# Indian Stock Market AI Prediction System
### Complete AI-Powered Stock Analysis & Prediction Platform

This notebook provides an interactive interface for:
- Training prediction models on Indian stocks (NSE)
- Predicting top stocks for February 2026
- Backtesting on historical months
- Analyzing model performance

In [ ]:
# Import libraries
from indian_stock_predictor import IndianStockPredictor
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')

print("✓ Libraries imported successfully")

## 1. Initialize Predictor

Choose your stock universe:

In [ ]:
# Option 1: Use full Nifty 50 (recommended for production)
predictor = IndianStockPredictor()

# Option 2: Use smaller set for faster testing
# predictor = IndianStockPredictor(
#     stock_list=['RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS', 
#                 'ICICIBANK.NS', 'HINDUNILVR.NS', 'ITC.NS', 'SBIN.NS']
# )

print(f"Total stocks: {len(predictor.stock_list)}")
print("\nStock list:")
for i, stock in enumerate(predictor.stock_list[:10], 1):
    print(f"{i}. {stock}")
if len(predictor.stock_list) > 10:
    print(f"... and {len(predictor.stock_list) - 10} more")

## 2. Data Collection & Feature Engineering

Fetch historical data and calculate technical indicators:

In [ ]:
# Define training period
end_date = datetime.now()
start_date = end_date - timedelta(days=1095)  # 3 years of data

print(f"Fetching data from {start_date.date()} to {end_date.date()}")
print("This may take a few minutes...\n")

# Build dataset
df = predictor.build_dataset(
    start_date=start_date,
    end_date=end_date,
    prediction_days=30  # Predict 30 days ahead
)

print(f"\nDataset shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"\nFeatures calculated: {len([col for col in df.columns if col not in ['Symbol', 'Target', 'Future_Return', 'Target_Multi']])}")

In [ ]:
# Preview the data
print("Sample data:")
df.head()

In [ ]:
# Data distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Target distribution
df['Target'].value_counts().plot(kind='bar', ax=axes[0, 0], color=['red', 'green'])
axes[0, 0].set_title('Target Distribution (Up vs Down)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Direction')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_xticklabels(['Down', 'Up'], rotation=0)

# Future returns distribution
df['Future_Return'].hist(bins=50, ax=axes[0, 1], color='steelblue', alpha=0.7)
axes[0, 1].set_title('Distribution of Future Returns', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Return')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=2)

# Stocks per symbol
symbol_counts = df['Symbol'].value_counts().head(15)
symbol_counts.plot(kind='barh', ax=axes[1, 0], color='coral')
axes[1, 0].set_title('Top 15 Stocks by Sample Count', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Number of Samples')

# RSI distribution
df['RSI'].hist(bins=30, ax=axes[1, 1], color='purple', alpha=0.7)
axes[1, 1].set_title('RSI Distribution', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('RSI Value')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].axvline(30, color='green', linestyle='--', label='Oversold')
axes[1, 1].axvline(70, color='red', linestyle='--', label='Overbought')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print(f"\nPositive cases: {df['Target'].sum()} ({df['Target'].mean()*100:.1f}%)")
print(f"Negative cases: {(df['Target']==0).sum()} ({(1-df['Target'].mean())*100:.1f}%)")

## 3. Model Training with Walk-Forward Validation

Train the model using time-series cross-validation:

In [ ]:
# Train model
model, fold_results = predictor.train_walk_forward(
    df, 
    prediction_days=30, 
    n_splits=5
)

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)

In [ ]:
# Visualize validation results
fold_nums = [r['fold'] for r in fold_results]
accuracies = [r['metrics']['accuracy'] for r in fold_results]
roc_aucs = [r['metrics']['roc_auc'] for r in fold_results]

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy across folds
ax[0].plot(fold_nums, accuracies, marker='o', linewidth=2, markersize=8, color='green')
ax[0].axhline(np.mean(accuracies), color='red', linestyle='--', label=f'Mean: {np.mean(accuracies):.3f}')
ax[0].set_xlabel('Fold', fontsize=12)
ax[0].set_ylabel('Accuracy', fontsize=12)
ax[0].set_title('Accuracy Across Folds', fontsize=14, fontweight='bold')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# ROC-AUC across folds
ax[1].plot(fold_nums, roc_aucs, marker='s', linewidth=2, markersize=8, color='blue')
ax[1].axhline(np.mean(roc_aucs), color='red', linestyle='--', label=f'Mean: {np.mean(roc_aucs):.3f}')
ax[1].set_xlabel('Fold', fontsize=12)
ax[1].set_ylabel('ROC-AUC', fontsize=12)
ax[1].set_title('ROC-AUC Across Folds', fontsize=14, fontweight='bold')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAverage Accuracy: {np.mean(accuracies):.3f} (+/- {np.std(accuracies):.3f})")
print(f"Average ROC-AUC: {np.mean(roc_aucs):.3f} (+/- {np.std(roc_aucs):.3f})")

In [ ]:
# Feature importance
import xgboost as xgb

feature_importance = pd.DataFrame({
    'feature': predictor.feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False).head(20)

plt.figure(figsize=(10, 8))
plt.barh(range(len(feature_importance)), feature_importance['importance'], color='steelblue')
plt.yticks(range(len(feature_importance)), feature_importance['feature'])
plt.xlabel('Importance Score', fontsize=12)
plt.title('Top 20 Most Important Features', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 4. Predictions for February 2026

Generate predictions for the target month:

In [ ]:
# Predict for February 2026
predictions_df = predictor.predict_stocks(target_month=2, target_year=2026)

print("\n" + "="*80)
print("TOP 10 STOCK PREDICTIONS FOR FEBRUARY 2026")
print("="*80)

top_10 = predictions_df.head(10)

for idx, row in top_10.iterrows():
    print(f"\n#{idx+1}. {row['Symbol']}")
    print(f"   Probability Up: {row['Probability_Up']*100:.1f}%")
    print(f"   Confidence: {row['Confidence_Score']*100:.1f}%")
    print(f"   Latest Price: ₹{row['Latest_Close']:.2f}")
    print(f"   Prediction: {row['Prediction']}")

In [ ]:
# Visualize top predictions
top_15 = predictions_df.head(15)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart of probabilities
colors = ['green' if p > 0.6 else 'orange' if p > 0.5 else 'red' 
          for p in top_15['Probability_Up']]
axes[0].barh(range(len(top_15)), top_15['Probability_Up'], color=colors)
axes[0].set_yticks(range(len(top_15)))
axes[0].set_yticklabels(top_15['Symbol'])
axes[0].set_xlabel('Probability of Rise', fontsize=12)
axes[0].set_title('Top 15 Stock Predictions', fontsize=14, fontweight='bold')
axes[0].axvline(0.5, color='black', linestyle='--', linewidth=1)
axes[0].set_xlim(0, 1)
axes[0].invert_yaxis()

# Confidence scores
axes[1].barh(range(len(top_15)), top_15['Confidence_Score'], color='steelblue')
axes[1].set_yticks(range(len(top_15)))
axes[1].set_yticklabels(top_15['Symbol'])
axes[1].set_xlabel('Confidence Score', fontsize=12)
axes[1].set_title('Prediction Confidence', fontsize=14, fontweight='bold')
axes[1].set_xlim(0.5, 1)
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Best stock highlight
best_stock = predictions_df.iloc[0]

print("\n" + "="*80)
print("🏆 BEST STOCK RECOMMENDATION FOR FEBRUARY 2026")
print("="*80)
print(f"\nTicker: {best_stock['Symbol']}")
print(f"Predicted Upside Probability: {best_stock['Probability_Up']*100:.1f}%")
print(f"Confidence Score: {best_stock['Confidence_Score']*100:.1f}%")
print(f"Latest Price: ₹{best_stock['Latest_Close']:.2f}")
print(f"Last Updated: {best_stock['Latest_Date']}")
print(f"\nPrediction: {best_stock['Prediction']}")
print(f"\n⚠️ Recommendation: Long position with appropriate risk management")
print(f"\n📊 Model analyzed {len(df)} data points across {len(predictor.stock_list)} stocks")
print(f"using {len(predictor.feature_columns)} technical and fundamental features.")

In [ ]:
# Save predictions
predictions_df.to_csv('predictions_feb_2026.csv', index=False)
print("\n✓ Predictions saved to: predictions_feb_2026.csv")

# Full predictions table
predictions_df.head(20)

## 5. Backtesting

Test the model on historical months to validate performance:

In [ ]:
# Backtest on recent months
test_months = [
    (12, 2025),
    (11, 2025),
    (10, 2025),
    (9, 2025),
    (8, 2025),
]

backtest_results = []

for month, year in test_months:
    print(f"\nBacktesting {month}/{year}...")
    result = predictor.backtest_month(month, year, prediction_days=30)
    
    if 'error' not in result:
        backtest_results.append(result)
        print(f"✓ Accuracy: {result['accuracy']*100:.1f}%")
    else:
        print(f"✗ {result['error']}")

print("\nBacktesting complete!")

In [ ]:
# Backtest summary
if backtest_results:
    summary = pd.DataFrame([
        {
            'Month': r['month'],
            'Accuracy': f"{r['accuracy']*100:.1f}%",
            'Total_Predictions': r['total_predictions'],
            'Top_Pick': r['top_pick']['symbol'],
            'Top_Pick_Return': f"{r['top_pick']['actual_return']*100:.2f}%",
            'Top_Pick_Correct': '✓' if r['top_pick']['correct'] else '✗',
            'Top_5_Avg_Return': f"{r['top_5_avg_return']*100:.2f}%"
        }
        for r in backtest_results
    ])
    
    print("\n" + "="*100)
    print("BACKTEST RESULTS SUMMARY")
    print("="*100)
    display(summary)
    
    # Calculate overall metrics
    avg_accuracy = np.mean([r['accuracy'] for r in backtest_results])
    top_pick_accuracy = np.mean([r['top_pick']['correct'] for r in backtest_results])
    avg_top5_return = np.mean([r['top_5_avg_return'] for r in backtest_results])
    
    print(f"\n📊 Overall Performance:")
    print(f"   Average Model Accuracy: {avg_accuracy*100:.1f}%")
    print(f"   Top Pick Success Rate: {top_pick_accuracy*100:.1f}%")
    print(f"   Average Top 5 Return: {avg_top5_return*100:.2f}%")

In [ ]:
# Visualize backtest performance
if backtest_results:
    months = [r['month'] for r in backtest_results]
    accuracies = [r['accuracy']*100 for r in backtest_results]
    top5_returns = [r['top_5_avg_return']*100 for r in backtest_results]
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    
    # Accuracy by month
    axes[0].bar(months, accuracies, color='steelblue', alpha=0.7)
    axes[0].axhline(50, color='red', linestyle='--', linewidth=2, label='Random (50%)')
    axes[0].axhline(np.mean(accuracies), color='green', linestyle='--', 
                    linewidth=2, label=f'Mean ({np.mean(accuracies):.1f}%)')
    axes[0].set_ylabel('Accuracy (%)', fontsize=12)
    axes[0].set_title('Model Accuracy by Month', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Top 5 returns by month
    colors = ['green' if r > 0 else 'red' for r in top5_returns]
    axes[1].bar(months, top5_returns, color=colors, alpha=0.7)
    axes[1].axhline(0, color='black', linewidth=1)
    axes[1].axhline(np.mean(top5_returns), color='blue', linestyle='--', 
                    linewidth=2, label=f'Mean ({np.mean(top5_returns):.2f}%)')
    axes[1].set_ylabel('Average Return (%)', fontsize=12)
    axes[1].set_xlabel('Month', fontsize=12)
    axes[1].set_title('Top 5 Picks Average Return by Month', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

## 6. Save Model & Results

In [ ]:
# Save the trained model
predictor.save_model('stock_predictor_model.pkl')

# Save backtest results
if backtest_results:
    summary.to_csv('backtest_summary.csv', index=False)
    print("✓ Backtest summary saved")

print("\n" + "="*80)
print("ALL FILES SAVED")
print("="*80)
print("\n1. stock_predictor_model.pkl - Trained model")
print("2. predictions_feb_2026.csv - February 2026 predictions")
print("3. backtest_summary.csv - Backtest results summary")

## Custom Backtesting

Test any specific month:

In [ ]:
# Backtest a specific month
custom_month = 1  # January
custom_year = 2026

custom_result = predictor.backtest_month(custom_month, custom_year, prediction_days=30)

if 'error' not in custom_result:
    print(f"\nBacktest Results for {custom_month}/{custom_year}:")
    print(f"Accuracy: {custom_result['accuracy']*100:.1f}%")
    print(f"Top Pick: {custom_result['top_pick']['symbol']}")
    print(f"Top Pick Return: {custom_result['top_pick']['actual_return']*100:.2f}%")
    print(f"Top 5 Average Return: {custom_result['top_5_avg_return']*100:.2f}%")
    
    # Show detailed predictions
    display(custom_result['all_predictions'].head(10))
else:
    print(f"Error: {custom_result['error']}")